In [1]:
!pip install transformers
!pip install lightning
# on mac
!pip3 install torch torchvision torchaudio

  Obtaining dependency information for lightning from https://files.pythonhosted.org/packages/f6/7d/fdd855f4ecf8525af8026097e433aeb90ebc965fe9f179b1d4900a83cba1/lightning-2.0.7-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 197.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 401.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 225.9 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for croniter<1.5.0,>=1.3.0 from https://files.pythonhosted.org/packages/f2/91/e5ae454da8200c6eb6cf94ca05d799b51e2cb2cc458a7737aebc0c5a21bb/croniter-1.4.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for deepdiff<8.0,>=5.7.0 from https://files.pythonhosted.org/packages/fe/b3/81bb598d24f1a48eaceb32243a91016385c0599196a59eaff6cd29299334/deepdiff-6.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for fastapi<2.0,>=0.92.0 from https://files.pythonhosted.or

In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():        
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
from torchmetrics.functional import accuracy, f1_score, precision, recall

/opt/homebrew/Caskroom/miniconda/base/envs/tesi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
davidson = pd.read_csv("datasets/davidson.csv")
davidson

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [4]:
davidson_classes = pd.read_csv("datasets/edited/davidson_with_classes.csv")
davidson_classes.head()

,care.virtue,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,positive,neutral,negative,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,1.0,0.0679,0.4113,0.5208,Partially Negative
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0326,0.4309,0.5365,Neutral but Negative Sentiment
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0031,0.0268,0.9701,Morally Negative
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0609,0.7310,0.2081,Neutral
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0119,0.1853,0.8027,Morally Negative


In [5]:
dataframe_davidson = pd.DataFrame(davidson['tweet']).join(davidson_classes['target'])
dataframe_davidson = dataframe_davidson.rename(columns={'tweet': 'text'})
dataframe_davidson.head()

,text,target
0,!!! RT @mayasolovely: As a woman you shouldn't...,Partially Negative
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Neutral but Negative Sentiment
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Morally Negative
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Neutral
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Morally Negative


In [6]:
mfrc = pd.read_csv("datasets/edited/mfrc_reduced.csv")
mfrc.head()

,text,subreddit,bucket,annotator,annotation,confidence,care.virtue,care.vice,fairness.virtue,fairness.vice,loyalty.virtue,loyalty.vice,authority.virtue,authority.vice,sanctity.virtue,sanctity.vice,cleaned_text
0,That particular part of the debate is especial...,europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,particular debate especially funny macron expl...
1,"/r/france is pretty lively, with it's own ling...",europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,r france pretty lively lingo usually delibera...
2,TBH Marion Le Pen would be better. Closet fasc...,neoliberal,French politics,annotator03,Non-Moral,Somewhat Confident,0,0,0,0,0,0,0,0,0,0,tbh marion le pen well closet fascist vs flamb...
3,it really is a very unusual situation isn't it...,europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,unusual situation fillon affair influence vote...
4,The Le Pen brand of conservatism and classical...,europe,French politics,annotator03,Authority,Somewhat Confident,0,0,0,0,0,0,0,0,0,0,le pen brand conservatism classical right wing...


In [7]:
mfrc_classes = pd.read_csv("datasets/edited/mfrc_with_classes.csv")
mfrc_classes.head()

,care.virtue,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,positive,neutral,negative,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.1067,0.3189,0.5743,Neutral but Negative Sentiment
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.1703,0.5590,0.2707,Neutral
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0658,0.4460,0.4882,Neutral but Negative Sentiment
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0204,0.3472,0.6324,Neutral but Negative Sentiment
4,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0455,0.7794,0.1751,Morally Positive


In [8]:
dataframe_mfrc = pd.DataFrame(mfrc['text']).join(mfrc_classes['target'])
dataframe_mfrc.head()

,text,target
0,That particular part of the debate is especial...,Neutral but Negative Sentiment
1,"/r/france is pretty lively, with it's own ling...",Neutral
2,TBH Marion Le Pen would be better. Closet fasc...,Neutral but Negative Sentiment
3,it really is a very unusual situation isn't it...,Neutral but Negative Sentiment
4,The Le Pen brand of conservatism and classical...,Morally Positive


In [9]:
df = pd.concat([dataframe_davidson, dataframe_mfrc]).reset_index()
del df['index']
df

,text,target
0,!!! RT @mayasolovely: As a woman you shouldn't...,Partially Negative
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Neutral but Negative Sentiment
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Morally Negative
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Neutral
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Morally Negative
...,...,...
42664,My job is actually ok and I work with loads of...,Neutral but Negative Sentiment
42665,Someone dying of a disease doesn't change that...,Morally Negative
42666,Some of these politicians and commentators tak...,Morally Negative
42667,"No, it's ""truth over facts! Wait where am I ag...",Neutral but Negative Sentiment


In [10]:
train_texts, val_texts = train_test_split(df, test_size=0.2)
train_texts.shape, val_texts.shape

((34135, 2), (8534, 2))

In [12]:
from utils.dataset_torch import CustomDataset
from transformers import BertTokenizer

train_dataset = CustomDataset(
  train_texts,
  BertTokenizer.from_pretrained('bert-base-uncased'),
  max_token_length=512
)

sample_item = train_dataset[0]
sample_item.keys()

TypeError: new(): invalid data type 'str'